# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [159]:
# TODO: Implement load the data here.
import pickle
training_file = './lab 2 data/train.p'
testing_file = './lab 2 data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [160]:
# TODO: Implement data normalization here.
import numpy as np
X_train=np.array(X_train,dtype=np.float32)
X_train=(X_train/255)-0.5

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [161]:
# TODO: Build a two-layer feedforward neural network with Keras here.
from keras.models import Sequential
from keras.layers import Activation,Dense

model=Sequential([
    Dense(128,input_shape=(3072,),name="hidden1"),
    Activation('relu'),
    Dense(128),
    Activation('relu'),
    Dense(43),
    Activation('softmax',name='output')
])


# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [163]:
# TODO: Compile and train the model here.
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

X_train=X_train.ravel()
X_train.shape=(-1,32*32*3)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history=model.fit(X_train,y_train,nb_epoch=2,batch_size=32)
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

Epoch 1/2
39209/39209 [==============================] - 6s - loss: 1.0624 - acc: 0.7045     
Epoch 2/2
39209/39209 [==============================] - 6s - loss: 0.8231 - acc: 0.7738     


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [112]:
# TODO: Split some of the training data into a validation dataset.
# TODO: Compile and train the model to measure validation accuracy.

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.23,
    random_state=832289)

valdata=(X_val,y_val)
history=model.fit(X_train,y_train,nb_epoch=2,batch_size=128,validation_data=valdata,shuffle=1)


# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % X_train.shape[0] / float(X_val.shape[0])
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 30190 samples, validate on 9019 samples
Epoch 1/2
30190/30190 [==============================] - 1s - loss: 0.9986 - acc: 0.7197 - val_loss: 0.9130 - val_acc: 0.7242
Epoch 2/2
30190/30190 [==============================] - 1s - loss: 0.8974 - acc: 0.7498 - val_loss: 0.7574 - val_acc: 0.7479


**Validation Accuracy**: (0.7242)

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [142]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
# TODO: Compile and train the model.
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

X_train=np.array(X_train,dtype=np.float32)
X_train=(X_train/255)-0.5

encoder = LabelBinarizer()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

from keras.layers import Flatten
from keras.layers import Convolution2D,MaxPooling2D
from keras.regularizers import l2,activity_l2

model=Sequential([
    Convolution2D(32,3,3,border_mode='valid',input_shape=(32,32,3),subsample=(1,1)),
    Flatten(),
    Dense(128,name="hidden1"),
    Activation('relu'),
    Dense(128),
    Activation('relu'),
    Dense(43),
    Activation('softmax',name='output')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.1,
    random_state=832289)

valdata=(X_val,y_val)
history=model.fit(X_train,y_train,nb_epoch=2,batch_size=64,validation_data=valdata)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 35288 samples, validate on 3921 samples
Epoch 1/2
35288/35288 [==============================] - 5s - loss: 0.7185 - acc: 0.8070 - val_loss: 0.2968 - val_acc: 0.9125
Epoch 2/2
35288/35288 [==============================] - 4s - loss: 0.1798 - acc: 0.9507 - val_loss: 0.2744 - val_acc: 0.9271


**Validation Accuracy**: (0.9125)

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [146]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# TODO: Compile and train the model.
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

X_train=np.array(X_train,dtype=np.float32)
X_train=(X_train/255)-0.5

encoder = LabelBinarizer()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

from keras.layers import Flatten
from keras.layers import Convolution2D,MaxPooling2D

model=Sequential([
    Convolution2D(32,3,3,border_mode='valid',input_shape=(32,32,3),subsample=(1,1)),
    Activation('relu'),
    MaxPooling2D((2,2),border_mode='valid'),
    Flatten(),
    Dense(128,name="hidden1"),
    Activation('relu'),
    Dense(128),
    Activation('relu'),
    Dense(43),
    Activation('softmax',name='output')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.1,
    random_state=832289)

valdata=(X_val,y_val)
history=model.fit(X_train,y_train,nb_epoch=1,batch_size=32,validation_data=valdata,shuffle=1)
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 35288 samples, validate on 3921 samples
Epoch 1/1
35288/35288 [==============================] - 8s - loss: 0.9489 - acc: 0.7258 - val_loss: 0.3010 - val_acc: 0.9169


**Validation Accuracy**: (0.9169)

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [147]:
# TODO: Re-construct the network and add dropout after the pooling layer.
# TODO: Compile and train the model.
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

X_train=np.array(X_train,dtype=np.float32)
X_train=(X_train/255)-0.5

encoder = LabelBinarizer()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

from keras.layers import Flatten
from keras.layers import Convolution2D,MaxPooling2D,Dropout

model=Sequential([
    Convolution2D(32,3,3,border_mode='valid',input_shape=(32,32,3),subsample=(1,1)),
    Activation('relu'),
    MaxPooling2D((2,2),border_mode='valid'),
    Dropout(0.5),
    Flatten(),
    Dense(128,name="hidden1"),
    Activation('relu'),
    Dense(128),
    Activation('relu'),
    Dense(43),
    Activation('softmax',name='output')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.1,
    random_state=832289)

valdata=(X_val,y_val)
history=model.fit(X_train,y_train,nb_epoch=1,batch_size=32,validation_data=valdata,shuffle=1)
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 35288 samples, validate on 3921 samples
Epoch 1/1
35288/35288 [==============================] - 8s - loss: 1.1221 - acc: 0.6784 - val_loss: 0.3554 - val_acc: 0.9016


In [151]:
**Validation Accuracy**: (0.9016)

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [187]:
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

X_train=np.array(X_train,dtype=np.float32)
X_train=(X_train/255)-0.5

encoder = LabelBinarizer()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

from keras.layers import Flatten
from keras.layers import Convolution2D,MaxPooling2D,Dropout
from keras.regularizers import l2,activity_l2

model=Sequential([
    Convolution2D(32,3,3,border_mode='valid',input_shape=(32,32,3),subsample=(1,1)),
    Activation('relu'),
    MaxPooling2D((2,2),border_mode='valid'),
    Convolution2D(32,3,3,border_mode='valid',input_shape=(32,32,3),subsample=(1,1)),
    Activation('relu'),
    MaxPooling2D((2,2),border_mode='valid'),
    #Dropout(0.5),
    Flatten(),
    Dense(128,name="hidden1"),
    Activation('relu'),
    Dense(43),
    Activation('softmax',name='output')
])

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    random_state=832289)

valdata=(X_val,y_val)
history=model.fit(X_train,y_train,nb_epoch=8,batch_size=64,validation_data=valdata,shuffle=1)

Train on 31367 samples, validate on 7842 samples
Epoch 1/8
31367/31367 [==============================] - 4s - loss: 1.6186 - acc: 0.5356 - val_loss: 0.7919 - val_acc: 0.7387
Epoch 2/8
31367/31367 [==============================] - 3s - loss: 0.3921 - acc: 0.8846 - val_loss: 0.7110 - val_acc: 0.7998
Epoch 3/8
31367/31367 [==============================] - 3s - loss: 0.1821 - acc: 0.9486 - val_loss: 0.1295 - val_acc: 0.9675
Epoch 4/8
31367/31367 [==============================] - 3s - loss: 0.1051 - acc: 0.9707 - val_loss: 0.6711 - val_acc: 0.8433
Epoch 5/8
31367/31367 [==============================] - 3s - loss: 0.0730 - acc: 0.9801 - val_loss: 0.0988 - val_acc: 0.9735
Epoch 6/8
31367/31367 [==============================] - 3s - loss: 0.0495 - acc: 0.9865 - val_loss: 0.0718 - val_acc: 0.9823
Epoch 7/8
31367/31367 [==============================] - 3s - loss: 0.0362 - acc: 0.9906 - val_loss: 0.0692 - val_acc: 0.9860
Epoch 8/8
31367/31367 [==============================] - 3s - loss: 0

**Best Validation Accuracy:** (0.99)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [188]:
with open('./lab 2 data/test.p', mode='rb') as f:
    test = pickle.load(f)

from keras.utils import np_utils
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

model.evaluate(X_test, Y_test)

12630/12630 [==============================] - 1s     


[0.42473223115894898, 0.92794932701808541]

**Test Accuracy:** (0.9279)

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.